In [413]:
import os
import numpy as np
import gym
import matplotlib.pyplot as plt
%matplotlib inline


In [414]:
tsp_data_txt = open('tsp_dataset.txt')

In [415]:
lines = tsp_data_txt.readlines()

In [416]:
tsp_data_txt = open('tsp_dataset.txt')
data = {}
lines = tsp_data_txt.readlines()
distance_matrix = np.zeros((len(lines),len(lines)))
for i,line in enumerate(lines):
    for j,element in enumerate((line.split('\n')[0].split('       '))):
        if(element == ''):
            continue
        
        distance_matrix[i][j-1] = int(element.strip())
data['distance_matrix']= distance_matrix
data['reward_matrix'] = [1 for i  in range(0,len(distance_matrix))]
data['depot'] = 0
data['n_state'] = len(distance_matrix)*len(distance_matrix)
data['n_action'] = len(distance_matrix)
data['visited_nodes'] = [0]


In [417]:
for i in range(0,len(data['distance_matrix'])):
    data['distance_matrix'][i][i] = 9999


In [418]:
def initialze_value(env):
    initial_P = 1/(env['n_action']-1)
    env['P'] = {}
    
    #P[state][action] = [(prob, next state, reward, done), ...]
    
    for i in range(0,env['n_state']):
        env['P'][i] ={}
        for j in range(0,env['n_action']):
            env['P'][i][j] = []
            for k in range(0,env['n_action']):
                ini_temp = [initial_P,k,data['reward_matrix'][k],False]
                env['P'][i][j].append(ini_temp)
            
    return env
    

In [419]:
def change_reward(envp):
    for i in range(0,envp['n_action']):
        for j in range(0,envp['n_action']):
            for k in range(0,envp['n_action']):
                if k==0:
                    env['P'][i][j][k][2]= -9999
                    env['P'][i][j][k][3]= True
                    break
    return envp

In [420]:
env = initialze_value(data)

In [421]:
for s in env['P'].keys():
    for a in env['P'][s]:
        for s_i,s_p in enumerate(env['P'][s][a]):
            if(s==a==s_i):
                env['P'][s][a][s_i][2] = -1

In [422]:
env['distance_matrix'][0]

array([9999.,   29.,   82.,   46.,   68.,   52.,   72.,   42.,   51.,
         55.,   29.,   74.,   23.,   72.,   46.])

In [423]:
def get_roots(v_list,env_dist=env['distance_matrix']):
    v_count = 14
    ini_point =0
    distance = 0
    for v_c in range(0,14):
        i = v_list[ini_point]
        j = v_list[ini_point+1]
        distance+=env['distance_matrix'][i][j]
        ini_point+=1
    return distance

In [424]:
Pi = np.random.uniform(size=(env['n_state'],env['n_action']))
Pi = Pi/np.sum(Pi,axis=1,keepdims=True)

In [425]:
def q_policy_evaluation(env,P,r,Pi,gamma=0.99,epsilon=1e-6):
    """
    Policy evaluation 
     env   : environment
     P     : state transition probability [S x A x S]
     r     : reward [S]
     Pi    : policy [S x A]
    returns
     Q     : Q-value [S x A]
    """
    # Extract environment information
    n_state = env['n_state']
    n_action = env['n_action']
    # Randomly initialize Q
    Q = np.random.uniform(size=(n_state,n_action))
    qe_count = 0
    while True:
        V = np.sum(Pi*Q,axis=1) # [S]
        
        V_tile = np.tile(V[np.newaxis,np.newaxis:],reps=(n_action,n_action,1)) # [S x A x S]
        V_tile = np.reshape(V_tile,(225,15,15))
        Q_prime = np.sum((r+gamma*V_tile)*P,axis=2) # [S x A]
        Q_dist = np.max(np.max(np.abs(Q-Q_prime)))
        #print(np.where(np.max(np.max(np.abs(Q-Q_prime)))==Q_dist)[0][0])
        Q = Q_prime
        
        #if(qe_count%10==0):
            #print(Q_dist)
        if(qe_count==100):
            break
        
        qe_count+=1
        
        if Q_dist < epsilon:
            break
    return Q


def q_policy_improvement(env,Q):
    """
    Policy improvement
     env   : environment
     Q     : Q-value [S x A]
    """
    # Extract environment information
    n_state = env['n_state']
    n_action = env['n_action']
    Pi = np.zeros((n_state,n_action))
    rm_lst = []
    
    while True:
        count=0
        for q_i,q_ in enumerate(Q):
            argmax = np.argmax(q_)
            if argmax not in rm_lst:
                rm_lst.append(argmax)
            if argmax in rm_lst:
                if(count%5==0):
                    Q[q_i][argmax] -= (1/env['distance_matrix'][q_i//15][argmax])*count
                count+=1
                #if count>20:
                    #Q[q_i][argmax] = -100
        if(len(rm_lst)==15):
            break
        rm_lst = []
    Pi[np.arange(n_state),np.argmax(Q,axis=1)] = 1 # Greedy policy update
    return Pi

def q_policy_iteration(env,gamma=0.99,epsilon=1e-6):
    """
    Policy iteration
    """
    q_count =0 
    n_state = env['n_state']
    n_action = env['n_action']
    # Random initial policy
    Pi = np.random.uniform(size=(n_state,n_action))
    Pi = Pi/np.sum(Pi,axis=1,keepdims=True)
    # Parse P and r from the environment
    P = np.zeros((n_state,n_action,n_action))
    r = np.zeros((n_state,n_action,n_action))
    for s in env['P'].keys(): # for all states s
        for a in env['P'][s].keys(): # for all actions a
            for prob,s_prime,reward,done in env['P'][s][a]:
                P[s][a][s_prime] = prob # model 
                r[s][a][s_prime] = (1/env['distance_matrix'][s//15][a])*reward
                #r[s][a][s_prime] = reward
    # Loop
    
    while True:
        Q = q_policy_evaluation(env,P,r,Pi,gamma=gamma,epsilon=epsilon) # policy evaluation
        
        Pi_prime = q_policy_improvement(env,Q) # policy improvement
        
        if (Pi == Pi_prime).all(): # if policy does not change
            break
        Pi = Pi_prime # update new policy
        break
    return Pi,Q,r,P
    #return r

In [426]:
Pi,Q,r,p = q_policy_iteration(env)

In [457]:
class MCAgent():
    def __init__(self,n_state,n_action,epsilon=1.0,alpha=0.1,gamma=0.995):
        """
        Initialize Monte Carlo Learning Agent 
        """
        self.n_state = n_state
        self.n_action = n_action
        self.epsilon = epsilon # epsilon greediness
        self.alpha = alpha # Q mixing rate
        self.gamma = gamma # discount factor
        
        # Initialize Q value
        self.Q = np.zeros([n_state,n_action])
        # Memory 
        self.samples = []
        
    def save_sample(self, state, action, reward, done): 
        """
        Save experience (s, a, r, done)
        """
        self.samples.append([state, action, reward, done])
        
    def update_Q(self):
        """
        Update Q
        """
        Q_old = self.Q # [S x A]
        g = 0
        G = Q_old
        for t in reversed(range(len(self.samples))): # for all samples in a reversed way
            state,action,reward,_ = self.samples[t]
            g = reward + self.gamma*g # g = r + gamma * g
            G[state][action] = g # update G
            
        # Update Q
        self.Q = Q_old + self.alpha*(G - Q_old)
        # Empty memory
        self.samples = []
        
    def update_epsilon(self,epsilon):
        self.epsilon = np.min([epsilon,1.0]) # decay
        
    def get_action(self,state):
        """
        Get action
        """
        if np.random.uniform() < self.epsilon: # random with epsilon probability 
            
            action = np.random.randint(0, high=self.n_action)
        else: # greedy action
            action = np.argmax(self.Q[state])
        return action

In [458]:
env = initialze_value(data)

In [473]:
n_state = env['n_state']
n_action = env['n_action']
M = MCAgent(n_state,n_action,epsilon=1,alpha=0.1,gamma=0.999)
envp = env['P']
n_episode = 1000

for e_idx in range(n_episode):
    rm_lst=[0]
    state = 0
    action = M.get_action(state)
    done = False
    
    
    while not done:
        if(len(rm_lst)==15):
            break
            
        if action not in rm_lst:
            rm_lst.append(action)
    
        next_state = action
        #print(state,action,state+action,envp[state][action][(state+action)//15])
        if action in rm_lst:
            reward=-100
            if e_idx>0:
                e_idx-=1
                
        next_action = M.get_action(next_state) # Get next action
        M.save_sample(state,action,reward,done) # Store samples
        state = next_state
        action = next_action
 
        
    # End of the episode
    M.update_Q() # Update Q value using sampled epsiode
    M.update_epsilon(100/(e_idx+1)) # Decaying epsilon
print ("MC learning done.")

MC learning done.


In [474]:
M.get_action(13)

0

In [475]:
b

3

In [476]:
np.random.uniform()

0.20889436331238465

In [477]:
rm_lst

[0, 5, 11, 1, 8, 3, 10, 13, 12, 14, 7, 9, 6, 4, 2]

In [201]:
class ImportanceSampling(MCAgent):
    def __init__(self, *args, **kwargs):
        super(ImportanceSampling, self).__init__(*args, **kwargs)
        self.C = np.zeros([self.n_state,self.n_action])
        self.b = self.set_behavior_policy(p_type='random')
        
    def set_behavior_policy(self, p_type):
        """
        Set behavior policy
        """
        if p_type == "random":
            def behavior_policy(state):
                prob = np.ones(self.n_action) / self.n_action
                return prob
        elif p_type == "greedy":
            def behavior_policy(state):
                prob = np.zeros(self.n_action)
                prob[np.argmax(state)] =1
                return prob
        return behavior_policy
            
            
    def update_Q(self):
        """
        Update Q
        """
        g = 0.
        for t in reversed(range(len(self.samples))): # for all samples in a reversed way
            state,action,reward,_ = self.samples[t]
            g = (reward + self.gamma*g) # g = (r + gamma * g) * 1/b
            self.C[state][action] += 1
            self.Q[state][action] += (1 / self.C[state][action]) * (g - self.Q[state][action])
            if action !=  np.argmax(self.Q[state]):
                break
            
        # Empty memory
        self.samples = []
                    
    def get_action(self, state):
        """
        Get action
        """
        probs = self.b(state)
        action = np.random.choice(np.arange(len(probs)), p=probs)
        return action

In [478]:
n_state = env['n_state']
n_action = env['n_action']
M = ImportanceSampling(n_state,n_action,epsilon=1.0,alpha=0.1,gamma=0.999)

envp = env['P']
n_episode = 1000

for e_idx in range(n_episode):
    rm_lst=[0]
    state = 0
    action = M.get_action(state)
    done = False
    
    
    while not done:
        if(len(rm_lst)==15):
            break
            
        if action not in rm_lst:
            rm_lst.append(action)
    
        next_state = action
        #print(state,action,state+action,envp[state][action][(state+action)//15])
        if action in rm_lst:
            reward=-100
            if e_idx>0:
                e_idx-=1
                
        next_action = M.get_action(next_state) # Get next action
        M.save_sample(state,action,reward,done) # Store samples
        state = next_state
        action = next_action
 
        
    # End of the episode
    M.update_Q() # Update Q value using sampled epsiode
    M.update_epsilon(100/(e_idx+1)) # Decaying epsilon
print ("MC learning done.")

MC learning done.


In [479]:
rm_lst

[0, 9, 4, 5, 7, 8, 3, 2, 12, 14, 10, 13, 6, 1, 11]

In [471]:
|

SyntaxError: invalid syntax (Temp/ipykernel_8172/525519296.py, line 1)

In [205]:
state

7